In [ ]:
# Import required libraries for model training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adamax

# Image data generator for train, test, valid dataframes
batch_size = 16      # defining the batch size
img_size = (224, 224)

training_generator = ImageDataGenerator()
testing_generator = ImageDataGenerator()

training_gen = training_generator.flow_from_dataframe(training_df, x_col='file_paths', y_col='classes', target_size=img_size,
                                    class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)

validation_gen = testing_generator.flow_from_dataframe(validation_df, x_col='file_paths', y_col='classes', target_size=img_size,
                                    class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)

testing_gen = testing_generator.flow_from_dataframe(testing_df, x_col='file_paths', y_col='classes', target_size=img_size,
                                    class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)

# Plotting few images from train generator
g_dict = training_gen.class_indices
classes = list(g_dict.keys())
images, labels = next(training_gen)

plt.figure(figsize=(20, 20))  # figure size

for i in range(12):  # number of images to show
    plt.subplot(4, 4, i + 1)
    image = images[i] / 255  # normalizing each image
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]
    plt.title(class_name, color='green', fontsize=12, weight="bold")
    plt.axis('off')
plt.tight_layout()
plt.show()

# Building CNN model for classification
img_shape = (224, 224, 3)        # Input image size dimension
class_count = len(list(training_gen.class_indices.keys()))
model = Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu", input_shape=img_shape),
    Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(256, activation="relu"),
    Dense(64, activation="relu"),
    Dense(class_count, activation="softmax")  # considering multiclass problem, softmax
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training the model
history = model.fit(training_gen, epochs=20, verbose=1, validation_data=validation_gen, shuffle=False)

# Plotting history of trained model
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']
valid_accuracy = history.history['val_accuracy']
valid_loss = history.history['val_loss']
index_loss = np.argmin(valid_loss)
valid_lowest = valid_loss[index_loss]
index_accuracy = np.argmax(valid_accuracy)
accuracy_highest = valid_accuracy[index_accuracy]

Epochs = [i + 1 for i in range(len(train_accuracy))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_accuracy + 1)}'

plt.figure(figsize=(20, 10))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)  # Plot for loss
plt.plot(Epochs, train_loss, 'g', label='Training_loss')
plt.plot(Epochs, valid_loss, 'r', label='Validation_loss')
plt.scatter(index_loss + 1, valid_lowest, s=150, c='blue', label=loss_label)
plt.title('Train & Valid Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)  # plot for accuracy
plt.plot(Epochs, train_accuracy, 'g', label='Training_Accuracy')
plt.plot(Epochs, valid_accuracy, 'r', label='Validation_Accuracy')
plt.scatter(index_accuracy + 1, accuracy_highest, s=150, c='blue', label=acc_label)
plt.title('Train & Valid Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
